# Análise de Campanha de Marketing Bancário
 Esse notebook faz uma análise de dados de uma campanha bancária,
usando **Python** e **SQL**.

## 1. Carregamento e preparação dos dados
Carregamos o arquivo CSV, tratamos colunas categóricas(transformando yes/no em 1/0) e transformamos o DataFrame em uma tabela SQLite chamada *bank* pra executar consultas SQL com mais praticidade.

In [216]:
import pandas as pd
import sqlite3

df = pd.read_csv('bank.csv', sep=';', encoding='utf-8')
for col in ['default', 'housing', 'loan', 'y']:
    df[col] = df[col].map({'yes':1 ,'no':0 ,'1':1, '0':0})

conn = sqlite3.connect('bank_marketing.db')
df.to_sql('bank', conn, if_exists='replace', index=False)

4521

## 2. Função run_query
Criamos essa função pra simplificar e facilitar as consultas.

In [217]:
def run_query(query):
    return pd.read_sql_query(query, conn)

## 3. Consultas SQL
Aqui, fazemos consultas para responder perguntas sobre o perfil dos clientes e os resultados da campanha.

### 3.1 Distribuição por profissão
Consulta para contar quantos clientes existem em cada profissão.
### 3.2 Média de saldo por estado civil
Consulta para identificar qual estado civil tem, em média, o maior saldo bancário.

In [218]:

q1 = run_query("""
SELECT job, COUNT(*) AS total
FROM bank
GROUP BY job
ORDER BY total DESC;
""")

q2 = run_query("""
SELECT marital, AVG(balance) AS media_saldo
FROM bank
GROUP BY marital
ORDER BY media_saldo DESC;
""")

print("Distribuição por profissão:")
display(q1)
print("Média de saldo por estado civil:")
display(q2)

Distribuição por profissão:


,job,total
0,management,969
1,blue-collar,946
2,technician,768
3,admin.,478
4,services,417
5,retired,230
6,self-employed,183
7,entrepreneur,168
8,unemployed,128
9,housemaid,112


Média de saldo por estado civil:


,marital,media_saldo
0,married,1463.195567
1,single,1460.414716
2,divorced,1122.390152


## 4. Conclusão
Resumo dos principais insights obtidos nas consultas.

In [219]:
q3 = run_query("""
SELECT housing, COUNT(*) AS total
FROM bank
GROUP BY housing;
""")

q4 = run_query("""
SELECT loan, COUNT(*) AS total
from bank
GROUP BY loan;
               """)

print("Número de empréstimos de habitação")
display(q3)
print("Número de empréstimos pessoais")
display(q4)

Número de empréstimos de habitação


,housing,total
0,0,1962
1,1,2559


Número de empréstimos pessoais


,loan,total
0,0,3830
1,1,691


In [220]:
q5 = run_query("""
SELECT y, COUNT(*) AS total
FROM bank
GROUP BY y;
""")

q6 = run_query("""
SELECT job, SUM(y) AS conversoes, COUNT(*) AS total,
ROUND(100.0 * SUM(y) / COUNT(*), 2) AS taxa_sucesso
FROM bank
GROUP BY job
ORDER BY taxa_sucesso DESC
""")

q7 = run_query("""
SELECT contact, SUM(y) AS conversoes, COUNT(*) AS total,
ROUND(100.0 * SUM(y) / COUNT(*), 2) AS taxa_sucesso
FROM bank
GROUP BY contact
ORDER BY taxa_sucesso DESC"""
)

print("Taxa de sucesso geral:")
display(q5)
print("Taxa de sucesso da campanha por profissão:")
display(q6)
print("Taxa de conversões por tipo de contato:")
display(q7)

Taxa de sucesso geral:


,y,total
0,0,4000
1,1,521


Taxa de sucesso da campanha por profissão:


,job,conversoes,total,taxa_sucesso
0,retired,54,230,23.48
1,student,19,84,22.62
2,unknown,7,38,18.42
3,management,131,969,13.52
4,housemaid,14,112,12.50
5,admin.,58,478,12.13
6,self-employed,20,183,10.93
7,technician,83,768,10.81
8,unemployed,13,128,10.16
9,services,38,417,9.11


Taxa de conversões por tipo de contato:


,contact,conversoes,total,taxa_sucesso
0,telephone,44,301,14.62
1,cellular,416,2896,14.36
2,unknown,61,1324,4.61


In [221]:
q8 = run_query("""
SELECT month, SUM(y) AS conversoes, COUNT(*) AS total,
ROUND (100.0 * SUM(y) / COUNT(*), 2) AS taxa_sucesso
FROM bank
GROUP BY month
ORDER BY taxa_sucesso"""
)

q9 = run_query("""
SELECT day, SUM(y) AS conversoes
FROM bank
GROUP BY day
ORDER BY conversoes DESC"""
)

print("Melhores meses pra conversão:")
display(q8)
print("Melhores dias do mês pra conversão:")
display(q9)

Melhores meses pra conversão:


,month,conversoes,total,taxa_sucesso
0,may,93,1398,6.65
1,jul,61,706,8.64
2,nov,39,389,10.03
3,jun,55,531,10.36
4,jan,16,148,10.81
5,aug,79,633,12.48
6,feb,38,222,17.12
7,apr,56,293,19.11
8,sep,17,52,32.69
9,mar,21,49,42.86


Melhores dias do mês pra conversão:


,day,conversoes
0,18,35
1,12,28
2,5,28
3,30,27
4,16,24
5,21,23
6,15,23
7,13,22
8,11,20
9,14,19


In [222]:
q10 = run_query("""
SELECT job, marital, ROUND(AVG(balance), 2) AS saldo_medio,
ROUND(AVG(age), 1) AS idade_media,
ROUND(100.0 * SUM(y) / COUNT(*), 2) AS taxa_sucesso
FROM bank
GROUP BY job, marital
ORDER BY taxa_sucesso DESC;
""")
print("Relação interessante entre trabalho, estado civil e idade")
display(q10)

Relação interessante entre trabalho, estado civil e idade


,job,marital,saldo_medio,idade_media,taxa_sucesso
0,retired,divorced,1771.47,64.0,37.21
1,self-employed,divorced,2572.07,48.3,33.33
2,housemaid,divorced,1166.23,47.5,30.77
3,student,single,1689.16,25.9,24.32
4,retired,married,2383.86,62.2,20.45
5,housemaid,single,4695.33,40.7,20.00
6,unknown,married,1275.93,51.2,20.00
7,entrepreneur,divorced,889.75,46.4,18.75
8,retired,single,3425.64,47.8,18.18
9,self-employed,single,1089.27,35.3,17.07
